In [75]:
import sklearn
import pathlib
import numpy as np
import scipy as sp
import pandas as pd
import spacy
import datetime

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [61]:
dataset_path = pathlib.Path('../../dataset/stocknews/')

In [62]:
data = pd.read_csv(dataset_path / 'Combined_News_DJIA.csv')

In [63]:
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [77]:
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] >= '2015-01-01']

# little cleaner for later use without redundancy

#concatenate all news
data["combined_news"] = data.filter(regex=("Top.*")).apply(lambda x: ''.join(str(x.values)), axis=1)
TRAIN_END = datetime.datetime(2014,12,31)
n_train = len(data[pd.to_datetime(data["Date"]) <= TRAIN_END])

X_train = data["combined_news"].values[:n_train]
y_train = data["Label"].values[:n_train]
X_test = data["combined_news"].values[n_train:]
y_test = data["Label"].values[n_train:]

print(X_train)

['[\'b"Georgia \\\'downs two Russian warplanes\\\' as countries move to brink of war"\'\n "b\'BREAKING: Musharraf to be impeached.\'"\n "b\'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)\'"\n "b\'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire\'"\n \'b"Afghan children raped with \\\'impunity,\\\' U.N. official says - this is sick, a three year old was raped and they do nothing"\'\n "b\'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.\'"\n \'b"Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO\\\'s side"\'\n \'b"The \\\'enemy combatent\\\' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it."\'\n "b\'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VID

 '[\'AirAsia flight found at the bottom of the Java Sea\'\n "North Korean defector details \'human experiments\' -- use of mentally and physically handicapped children in chemical weapons tests \'the last straw\'"\n \'Korean Air ex-executive Cho Hyun-ah arrested - earlier she ordered a plane to turn back on the runway in New York after nuts were served in a bag, not on a plate\'\n \'South Korean to drop Sony film "The Interview" in North Korea by balloon\'\n \'U.S. opening of oil export widens battle: The Obama administration has opened a new front in the global battle for oil market share, effectively clearing the way for the shipment of as much as a million barrels per day of ultra-light U.S. crude to the rest of the world\'\n \'FBI agents investigating the Sony Pictures hack were briefed Monday by a security firm that says its research points to laid-off Sony staff, not North Korea, as the prepatrator\'\n \'Indian Government Orders Blocking of 32 Websites Including GitHub, Archive.o

In [65]:
def get_headlines(data):
    headlines = []
    for row in range(0,len(data.index)):
        headlines.append(' '.join(str(x) for x in data.iloc[row,2:27]))
    
    return headlines

# BASIC PIPELINE

In [82]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(2, 3), max_features=10_000)
reducer = TruncatedSVD(n_components=5000)
normalizer = Normalizer()
clf = LogisticRegression(solver='lbfgs')


model = Pipeline([('vectorizer', vectorizer),('reducer', reducer),('normalizer', normalizer),('clf',clf)])
model.fit(X_train, y_train)

preds = model.predict(X_test)
print(accuracy_score(y_test, preds))
print(roc_auc_score(y_true = y_test, y_score = preds))

0.5317460317460317
0.5288138440860215


# MODEL 1

In [83]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [84]:
trainheadlines[0]

'b"Georgia \'downs two Russian warplanes\' as countries move to brink of war" b\'BREAKING: Musharraf to be impeached.\' b\'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)\' b\'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire\' b"Afghan children raped with \'impunity,\' U.N. official says - this is sick, a three year old was raped and they do nothing" b\'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.\' b"Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO\'s side" b"The \'enemy combatent\' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." b\'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]\' b\'Did the U.S. Prep Georgia for War with Russia?\'

In [86]:
basicvectorizer = CountVectorizer()
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)
# print(basictrain[0])

(1611, 31675)


In [90]:
basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(basictrain, train["Label"])

In [91]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
basictest = basicvectorizer.transform(testheadlines)
predictions = basicmodel.predict(basictest)

In [105]:
print(pd.crosstab(test["Label"], predictions, rownames=["Actual"], colnames=["Predicted"]))
print(sklearn.metrics.classification_report(y_test, predictions))

Predicted   0    1
Actual            
0          61  125
1          92  100
              precision    recall  f1-score   support

           0       0.40      0.33      0.36       186
           1       0.44      0.52      0.48       192

   micro avg       0.43      0.43      0.43       378
   macro avg       0.42      0.42      0.42       378
weighted avg       0.42      0.43      0.42       378



# MODEL 2

In [100]:
advancedvectorizer = CountVectorizer(ngram_range=(3,3))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [102]:
advancedmodel = LogisticRegression(solver='lbfgs')
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])

In [103]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
advpredictions = advancedmodel.predict(advancedtest)

In [104]:
print(sklearn.metrics.classification_report(test['Label'], advpredictions))

              precision    recall  f1-score   support

           0       0.50      0.03      0.05       186
           1       0.51      0.97      0.67       192

   micro avg       0.51      0.51      0.51       378
   macro avg       0.50      0.50      0.36       378
weighted avg       0.50      0.51      0.36       378



# Deep

In [106]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [107]:
advancedvectorizer = CountVectorizer(ngram_range=(1,1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [108]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)

In [109]:
advancedtrain.shape

(1611, 31675)

In [32]:
advancedtest.shape

(378, 31675)

# Keras model

In [110]:
import keras

In [111]:
model = keras.models.Sequential()

model.add(keras.layers.Dense(16, activation='sigmoid', input_shape=(advancedtrain.shape[1],)))
model.add(keras.layers.Dropout(0.9))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dropout(0.9))
model.add(keras.layers.Dense(4, activation='relu'))

model.add(keras.layers.Dense(1, activation='sigmoid'))

In [112]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [113]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 16)                506816    
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 136       
_________________________________________________________________
dropout_4 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 5         
Total params: 506,993
Trainable params: 506,993
Non-trainable params: 0
_________________________________________________________________


In [114]:
model.fit(
    advancedtrain,
    train['Label'],
    batch_size=100,
    epochs=15,
    validation_data=(advancedtest, test['Label'])
)

Train on 1611 samples, validate on 378 samples
Epoch 1/15
1611/1611 [==============================] - ETA: 5:49 - loss: 0.8972 - acc: 0.410 - ETA: 1:41 - loss: 1.0349 - acc: 0.473 - ETA: 51s - loss: 1.0240 - acc: 0.476 - ETA: 30s - loss: 1.0076 - acc: 0.49 - ETA: 18s - loss: 1.0630 - acc: 0.48 - ETA: 10s - loss: 1.0661 - acc: 0.50 - ETA: 5s - loss: 1.0239 - acc: 0.5100 - ETA: 1s - loss: 0.9942 - acc: 0.510 - 30s 19ms/step - loss: 0.9841 - acc: 0.5071 - val_loss: 0.6932 - val_acc: 0.5079
Epoch 2/15
1611/1611 [==============================] - ETA: 0s - loss: 0.8339 - acc: 0.550 - ETA: 0s - loss: 0.7642 - acc: 0.540 - ETA: 0s - loss: 0.7748 - acc: 0.532 - ETA: 0s - loss: 0.7666 - acc: 0.528 - ETA: 0s - loss: 0.7549 - acc: 0.521 - ETA: 0s - loss: 0.7366 - acc: 0.532 - ETA: 0s - loss: 0.7442 - acc: 0.529 - ETA: 0s - loss: 0.7409 - acc: 0.533 - ETA: 0s - loss: 0.7354 - acc: 0.536 - 1s 498us/step - loss: 0.7345 - acc: 0.5307 - val_loss: 0.6933 - val_acc: 0.5079
Epoch 3/15
1611/1611 [=======

# Binary classifier model - why is this the title?!

In [115]:
import spacy

In [121]:
def cleaning(doc):
    txt = [token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)

In [127]:
nlp = spacy.load('en', disable=['ner', 'parser'])

In [128]:
trainheadlines = []
labelstrain = []
for row in range(0,len(train.index)):
    trainheadlines.extend(str(x) for x in train.iloc[row,2:27])
    labelstrain.extend( [train.iloc[row, 1],]* 25 )

In [129]:
testheadlines = []
labelstest = []
for row in range(0,len(test.index)):
    testheadlines.extend(str(x) for x in test.iloc[row,2:27])
    labelstest.extend( [test.iloc[row, 1],] * 25 )

In [130]:
import re

brief_cleaning = (re.sub("[^A-Za-z\"]+", ' ', str(row)).lower() for row in trainheadlines)

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

In [131]:
txt[:10]

['b"georgia down russian warplane country brink war "',
 'b break musharraf impeach',
 'b russia today column troop roll south ossetia footage fight youtube',
 'b russian tank move capital south ossetia reportedly completely destroy georgian artillery fire',
 'b"afghan child rape impunity u n official say sick year old rape "',
 'b russian tank enter south ossetia whilst georgia shoot russian jet',
 'b"breake georgia invade south ossetia russia warn intervene s "',
 'b"the enemy combatent trial sham salim haman sentence year keep longer feel like "',
 'b georgian troop retreat s osettain capital presumably leave people kill video',
 'b u s prep georgia war russia']

In [132]:
# advancedvectorizer = CountVectorizer(ngram_range=(2,2))
advancedvectorizer = CountVectorizer()
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
advancedtest = advancedvectorizer.transform(testheadlines)

In [133]:
labelstrain = np.array(labelstrain)
labelstest = np.array(labelstest)

In [134]:
print(advancedtrain.shape)
print(advancedtest.shape)
print(labelstrain.shape)
print(labelstest.shape)

(40275, 31675)
(9450, 31675)
(40275,)
(9450,)


In [135]:
import keras

In [136]:
model = keras.models.Sequential()

model.add(keras.layers.Dense(16, activation='sigmoid', input_shape=(advancedtrain.shape[1],)))
model.add(keras.layers.Dropout(0.9))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.9))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.9))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.9))

model.add(keras.layers.Dense(1, activation='sigmoid'))

In [137]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [138]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 16)                506816    
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                544       
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 32)                1056      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)                1056      
__________

In [139]:
model.fit(
    advancedtrain,
    labelstrain,
    batch_size=100,
    epochs=5,
    validation_data=(advancedtest, labelstest)
)

Train on 40275 samples, validate on 9450 samples
Epoch 1/5
40275/40275 [==============================] - ETA: 2:24:53 - loss: 2.2677 - acc: 0.41 - ETA: 48:13 - loss: 2.4314 - acc: 0.5000 - ETA: 28:52 - loss: 2.5026 - acc: 0.49 - ETA: 20:35 - loss: 2.6527 - acc: 0.49 - ETA: 15:59 - loss: 2.5830 - acc: 0.51 - ETA: 13:03 - loss: 2.5698 - acc: 0.50 - ETA: 11:01 - loss: 2.5176 - acc: 0.51 - ETA: 9:32 - loss: 2.5305 - acc: 0.5140 - ETA: 8:24 - loss: 2.4749 - acc: 0.513 - ETA: 7:30 - loss: 2.4290 - acc: 0.520 - ETA: 6:46 - loss: 2.4927 - acc: 0.517 - ETA: 6:10 - loss: 2.4785 - acc: 0.519 - ETA: 5:40 - loss: 2.4807 - acc: 0.518 - ETA: 5:14 - loss: 2.4678 - acc: 0.519 - ETA: 4:52 - loss: 2.4581 - acc: 0.517 - ETA: 4:32 - loss: 2.4546 - acc: 0.518 - ETA: 4:15 - loss: 2.4469 - acc: 0.521 - ETA: 4:00 - loss: 2.4651 - acc: 0.517 - ETA: 3:47 - loss: 2.4567 - acc: 0.518 - ETA: 3:34 - loss: 2.4458 - acc: 0.519 - ETA: 3:23 - loss: 2.4114 - acc: 0.522 - ETA: 3:13 - loss: 2.4340 - acc: 0.520 - ETA: 3:04

40275/40275 [==============================] - ETA: 16s - loss: 1.2090 - acc: 0.55 - ETA: 15s - loss: 1.1609 - acc: 0.57 - ETA: 15s - loss: 1.2698 - acc: 0.56 - ETA: 15s - loss: 1.2007 - acc: 0.56 - ETA: 14s - loss: 1.1584 - acc: 0.55 - ETA: 14s - loss: 1.1692 - acc: 0.54 - ETA: 14s - loss: 1.1413 - acc: 0.54 - ETA: 14s - loss: 1.1310 - acc: 0.54 - ETA: 14s - loss: 1.1197 - acc: 0.54 - ETA: 14s - loss: 1.1221 - acc: 0.53 - ETA: 14s - loss: 1.1027 - acc: 0.54 - ETA: 14s - loss: 1.0911 - acc: 0.54 - ETA: 14s - loss: 1.0965 - acc: 0.54 - ETA: 13s - loss: 1.0912 - acc: 0.54 - ETA: 13s - loss: 1.0749 - acc: 0.54 - ETA: 13s - loss: 1.0584 - acc: 0.54 - ETA: 13s - loss: 1.0603 - acc: 0.54 - ETA: 13s - loss: 1.0534 - acc: 0.54 - ETA: 13s - loss: 1.0430 - acc: 0.54 - ETA: 13s - loss: 1.0338 - acc: 0.54 - ETA: 13s - loss: 1.0292 - acc: 0.54 - ETA: 13s - loss: 1.0269 - acc: 0.54 - ETA: 13s - loss: 1.0329 - acc: 0.53 - ETA: 13s - loss: 1.0387 - acc: 0.53 - ETA: 13s - loss: 1.0282 - acc: 0.53 - ETA

40275/40275 [==============================] - ETA: 14s - loss: 0.7355 - acc: 0.55 - ETA: 13s - loss: 0.7116 - acc: 0.56 - ETA: 14s - loss: 0.7579 - acc: 0.54 - ETA: 14s - loss: 0.7456 - acc: 0.52 - ETA: 14s - loss: 0.7456 - acc: 0.53 - ETA: 14s - loss: 0.7407 - acc: 0.52 - ETA: 14s - loss: 0.7441 - acc: 0.53 - ETA: 14s - loss: 0.7457 - acc: 0.53 - ETA: 14s - loss: 0.7441 - acc: 0.53 - ETA: 14s - loss: 0.7438 - acc: 0.53 - ETA: 14s - loss: 0.7412 - acc: 0.53 - ETA: 13s - loss: 0.7354 - acc: 0.53 - ETA: 13s - loss: 0.7327 - acc: 0.54 - ETA: 13s - loss: 0.7302 - acc: 0.54 - ETA: 13s - loss: 0.7294 - acc: 0.54 - ETA: 13s - loss: 0.7301 - acc: 0.54 - ETA: 13s - loss: 0.7320 - acc: 0.54 - ETA: 13s - loss: 0.7303 - acc: 0.54 - ETA: 13s - loss: 0.7317 - acc: 0.54 - ETA: 13s - loss: 0.7335 - acc: 0.54 - ETA: 13s - loss: 0.7333 - acc: 0.54 - ETA: 13s - loss: 0.7316 - acc: 0.54 - ETA: 13s - loss: 0.7336 - acc: 0.54 - ETA: 12s - loss: 0.7347 - acc: 0.54 - ETA: 12s - loss: 0.7337 - acc: 0.54 - ETA

40275/40275 [==============================] - ETA: 14s - loss: 0.6963 - acc: 0.57 - ETA: 14s - loss: 0.6937 - acc: 0.56 - ETA: 14s - loss: 0.6923 - acc: 0.55 - ETA: 14s - loss: 0.7008 - acc: 0.54 - ETA: 14s - loss: 0.7020 - acc: 0.54 - ETA: 14s - loss: 0.7010 - acc: 0.53 - ETA: 14s - loss: 0.6957 - acc: 0.55 - ETA: 13s - loss: 0.6941 - acc: 0.55 - ETA: 13s - loss: 0.6961 - acc: 0.55 - ETA: 13s - loss: 0.6971 - acc: 0.55 - ETA: 13s - loss: 0.6997 - acc: 0.54 - ETA: 13s - loss: 0.6996 - acc: 0.54 - ETA: 13s - loss: 0.7000 - acc: 0.54 - ETA: 13s - loss: 0.6994 - acc: 0.54 - ETA: 13s - loss: 0.6997 - acc: 0.54 - ETA: 13s - loss: 0.6985 - acc: 0.54 - ETA: 13s - loss: 0.6989 - acc: 0.54 - ETA: 13s - loss: 0.6985 - acc: 0.54 - ETA: 13s - loss: 0.6979 - acc: 0.54 - ETA: 12s - loss: 0.6985 - acc: 0.54 - ETA: 12s - loss: 0.6980 - acc: 0.54 - ETA: 12s - loss: 0.6977 - acc: 0.54 - ETA: 12s - loss: 0.6977 - acc: 0.53 - ETA: 12s - loss: 0.6974 - acc: 0.53 - ETA: 12s - loss: 0.6977 - acc: 0.54 - ETA

40275/40275 [==============================] - ETA: 16s - loss: 0.7010 - acc: 0.51 - ETA: 15s - loss: 0.6968 - acc: 0.52 - ETA: 15s - loss: 0.6995 - acc: 0.52 - ETA: 15s - loss: 0.6978 - acc: 0.51 - ETA: 15s - loss: 0.6952 - acc: 0.52 - ETA: 15s - loss: 0.6941 - acc: 0.53 - ETA: 15s - loss: 0.6938 - acc: 0.53 - ETA: 15s - loss: 0.6934 - acc: 0.53 - ETA: 15s - loss: 0.6930 - acc: 0.53 - ETA: 15s - loss: 0.6938 - acc: 0.53 - ETA: 15s - loss: 0.6943 - acc: 0.52 - ETA: 14s - loss: 0.6938 - acc: 0.52 - ETA: 14s - loss: 0.6935 - acc: 0.52 - ETA: 14s - loss: 0.6935 - acc: 0.53 - ETA: 14s - loss: 0.6939 - acc: 0.52 - ETA: 14s - loss: 0.6933 - acc: 0.52 - ETA: 14s - loss: 0.6929 - acc: 0.53 - ETA: 14s - loss: 0.6929 - acc: 0.53 - ETA: 14s - loss: 0.6929 - acc: 0.53 - ETA: 14s - loss: 0.6920 - acc: 0.53 - ETA: 14s - loss: 0.6924 - acc: 0.53 - ETA: 14s - loss: 0.6930 - acc: 0.53 - ETA: 13s - loss: 0.6935 - acc: 0.53 - ETA: 13s - loss: 0.6932 - acc: 0.53 - ETA: 13s - loss: 0.6929 - acc: 0.53 - ETA